<a href="https://colab.research.google.com/github/BlueGod01/Recommendation_Engine_GENAI/blob/main/Recomendation_Engine_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install langchain huggingface_hub transformers sqlalchemy

  Using cached langchain-0.2.11-py3-none-any.whl.metadata (7.1 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.3/990.3 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 377.6/377.6 kB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.9/139.9 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 10.8 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0


In [6]:
from langchain.chains import LLMChain
from transformers import pipeline
from sqlalchemy import create_engine, MetaData, Table, select

In [7]:
def get_llm_model():
    model_name = "huggingface/llama3"  # Replace with the actual Llama3 model name
    llm = HuggingFaceLLM.from_pretrained(model_name)
    return llm

In [8]:
def connect_to_database(db_url):
    engine = create_engine(db_url)
    connection = engine.connect()
    metadata = MetaData()
    return connection, metadata

In [9]:
def fetch_course_data(connection, metadata, topic, learner_level, quiz_result):
    courses = Table('courses', metadata, autoload_with=connection)
    query = select([courses.c.name, courses.c.url, courses.c.description, courses.c.tutor]).where(
        courses.c.topic == topic,
        courses.c.level == learner_level,
        courses.c.minimum_score <= quiz_result
    )
    result = connection.execute(query)
    return result.fetchall()

In [10]:
def generate_course_suggestions(topic, learner_level, quiz_result):
    # Connect to the database
    db_url = 'your_database_connection_string'
    connection, metadata = connect_to_database(db_url)

    # Fetch course data
    course_data = fetch_course_data(connection, metadata, topic, learner_level, quiz_result)

    if not course_data:
        return "No suitable courses found."

    # Use LLM to refine suggestions
    llm = get_llm_model()

    courses_info = []
    for course in course_data:
        course_info = {
            "Course Name": course.name,
            "Course URL": course.url,
            "Brief": llm(f"Provide a brief about the course '{course.name}' and its tutor '{course.tutor}'. Description: {course.description}")
        }
        courses_info.append(course_info)

    return courses_info


In [12]:
def suggest_courses(topic, learner_level, quiz_result):
    suggestions = generate_course_suggestions(topic, learner_level, quiz_result)
    for suggestion in suggestions:
        print(f"Course Name: {suggestion['Course Name']}")
        print(f"Course URL: {suggestion['Course URL']}")
        print(f"Brief: {suggestion['Brief']}\n")